In [10]:
import numpy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [25]:
# ESPECIFICANDO QUAL DATASET SERÁ UTILIZADO
bits = 16
amostragem = 48

# CRIANDO O CAMINHO PARA O CSV
caminhoCSV = "../datasets/features_" + str(bits) + "bits_" + str(amostragem) + "k.csv"

# ABRINDO O CSV COMO UM NUMPY ARRAY
datasetOriginal = numpy.genfromtxt(caminhoCSV, delimiter=",")

# DELETANDO A PRIMEIRA LINHA (CABECALHO) E A SEGUNDA COLUNA (NOMES DOS ARQUIVOS)
datasetOriginal = numpy.delete(datasetOriginal, 0, 0)
datasetOriginal = numpy.delete(datasetOriginal, 1, 1)

In [26]:
# VISUALIZANDO ALGUM DADO
dadoAleatorio = 100

print("[0] PASTA:\t\t", datasetOriginal[dadoAleatorio][0])
print("[1] MFCC0:\t\t", datasetOriginal[dadoAleatorio][1])
print("[2] MFCC1:\t\t", datasetOriginal[dadoAleatorio][2])
print("[3] MFCC2:\t\t", datasetOriginal[dadoAleatorio][3])
print("[4] MFCC3:\t\t", datasetOriginal[dadoAleatorio][4])
print("[5] MFCC4:\t\t", datasetOriginal[dadoAleatorio][5])
print("[6] MFCC5:\t\t", datasetOriginal[dadoAleatorio][6])
print("[7] MFCC6:\t\t", datasetOriginal[dadoAleatorio][7])
print("[8] MFCC7:\t\t", datasetOriginal[dadoAleatorio][8])
print("[9] MFCC8:\t\t", datasetOriginal[dadoAleatorio][9])
print("[10] MFCC9:\t\t", datasetOriginal[dadoAleatorio][10])
print("[11] MFCC10:\t\t", datasetOriginal[dadoAleatorio][11])
print("[12] MFCC11:\t\t", datasetOriginal[dadoAleatorio][12])
print("[13] MFCC12:\t\t", datasetOriginal[dadoAleatorio][13])
print("[14] RMS:\t\t", datasetOriginal[dadoAleatorio][14])
print("[15] CENTROIDE:\t\t", datasetOriginal[dadoAleatorio][15])
print("[16] LARGURA:\t\t", datasetOriginal[dadoAleatorio][16])
print("[17] PLANICIDADE:\t", datasetOriginal[dadoAleatorio][17])
print("[18] ROLLOFF:\t\t", datasetOriginal[dadoAleatorio][18])
print("[19] ZCR:\t\t", datasetOriginal[dadoAleatorio][19])
print("[20] ASSIMETRIA:\t", datasetOriginal[dadoAleatorio][20])
print("[21] CURTOSE:\t\t", datasetOriginal[dadoAleatorio][21])
print("[22] VARIANCIA:\t\t", datasetOriginal[dadoAleatorio][22])
print("[23] CLASSIFICACAO:\t", datasetOriginal[dadoAleatorio][23])

[0] PASTA:		 1.0
[1] MFCC0:		 -226.54393747844244
[2] MFCC1:		 140.98918142663464
[3] MFCC2:		 -43.09198176972208
[4] MFCC3:		 54.58026594224066
[5] MFCC4:		 -21.203846110848318
[6] MFCC5:		 14.664003033541944
[7] MFCC6:		 -12.983213525002245
[8] MFCC7:		 15.706151689337831
[9] MFCC8:		 -10.612478133380957
[10] MFCC9:		 10.136786853547216
[11] MFCC10:		 -1.6929493287185677
[12] MFCC11:		 2.862301608037136
[13] MFCC12:		 -4.607714719751819
[14] RMS:		 0.03107336
[15] CENTROIDE:		 3517.7396779161822
[16] LARGURA:		 3714.066105297125
[17] PLANICIDADE:	 0.00586569
[18] ROLLOFF:		 7115.587599734043
[19] ZCR:		 0.08398956948138298
[20] ASSIMETRIA:	 0.013166787102818489
[21] CURTOSE:		 0.8388823726512014
[22] VARIANCIA:		 0.0009904432
[23] CLASSIFICACAO:	 7.0


In [27]:
def kFold(dataset):

    # ESSA FUNCAO RETORNARA UMA MATRIZ COM AS SEGUINTES DIMENSOES
    # matriz[iteracaoKFold][colunaDeTreinoOuTeste][dadoPuro][feature]
    # iteracaoKFold -> vai de 0 a 9 e representa as 10 iteracoes do KFold
    # colunaDeTreinoOuTeste -> vai pegar os dados de treinamento (0) ou os de teste (1)
    # dadoPuro -> dado com varias features, a pasta e a classificacao correta
    # feature -> seleciona uma das 24 features do dado
    
    matriz = []
    
    for pastaTeste in range(1,11):

        rodada = []
        xTrain = []
        xTest  = []

        for dado in dataset:
            if int(dado[0]) == pastaTeste:
                xTest.append(dado)
            else:
                xTrain.append(dado)
        
        rodada.append(xTrain)
        rodada.append(xTest)
        
        matriz.append(rodada)
    
    return matriz

In [28]:
def rearranjarDataset(dataset, datasetOriginal, featureAtual, i):
    
    # ESSA FUNCAO ADICIONA UMA NOVA FEATURE AO DATASET
    
    # REMOVENDO A COLUNA DE CLASSIFICACAO
    dataset = dataset[:,0:-1]

    # SE NAO FOR A PRIMEIRA ITERACAO, EXCLUIR A ULTIMA FEATURE
    if(i != 0):
        dataset = dataset[:,0:-1]

    # ADICIONANDO A FEATURE ATUAL NO DATASET
    colunaNova = datasetOriginal[:,featureAtual]
    dataset = numpy.column_stack((dataset, colunaNova))

    # ADICIONANDO NOVAMENTE A CLASSIFICACAO
    colunaNova = datasetOriginal[:,23]
    dataset = numpy.column_stack((dataset, colunaNova))
    
    return dataset

In [29]:
def novaMelhorFeature(dataset, datasetOriginal, melhorFeature, featuresRestantes):
    
    # ESSA FUNCAO SERVE PARA QUE, DEPOIS DE DEFINIDA A NOVA FEATURE QUE VAI ENTRAR NO DATASET PERMANENTEMENTE, 
    # REARRANJAR O DATASET E REMOVER ESSA FEATURE DO ARRAY DE FEATURES RESTANTES
    
    # REMOVENDO A COLUNA DE CLASSIFICACAO E A ULTIMA FEATURE UTILIZADA
    dataset = dataset[:,0:-2]

    # ADICIONANDO A FEATURE NOVA NO DATASET
    colunaNova = datasetOriginal[:,melhorFeature]
    dataset = numpy.column_stack((dataset, colunaNova))

    # ADICIONANDO NOVAMENTE A CLASSIFICACAO
    colunaNova = datasetOriginal[:,23]
    dataset = numpy.column_stack((dataset, colunaNova))
    
    # REMOVENDO DO ARRAY DE FEATURES RESTANTES
    indexPraRemover = numpy.where(featuresRestantes == melhorFeature)
    featuresRestantes = numpy.delete(featuresRestantes, indexPraRemover)
        
    return dataset, featuresRestantes

In [40]:
# OBTENDO O DATASET
dataset = datasetOriginal

# CRIANDO O CLASSIFICADOR
knn = KNeighborsClassifier(7)

# PARA CADA ITERACAO DO KFOLD
arrayAcuraciasKNN = []
matrizKFold = kFold(dataset)
for iteracaoKFold in matrizKFold:

    # SEPARANDO OS DADOS DE TREINAMENTO E TESTE E JÁ EXCLUINDO A COLUNA 0 (pasta)
    dadosTreino = numpy.delete(iteracaoKFold[0], 0, axis=1)
    dadosTeste  = numpy.delete(iteracaoKFold[1], 0, axis=1)

    # SEPARANDO O QUE E X E Y
    xTrain = numpy.delete(dadosTreino, 22, axis=1) # exclui a coluna do target
    xTest  = numpy.delete(dadosTeste, 22, axis=1) # exclui a coluna do target
    yTrain = numpy.delete(dadosTreino, numpy.s_[0:22], axis=1).ravel() # exclui as colunas menos a 22
    yTest  = numpy.delete(dadosTeste, numpy.s_[0:22], axis=1).ravel() # exclui as colunas menos a 22

    # TREINANDO O CLASSIFICADOR (ignorando a pasta e usando a coluna 23 como target)
    knn = knn.fit(xTrain, yTrain)

    # PREDIZENDO
    yKNN = knn.predict(xTest)

    # VERIFICANDO A ACURACIA
    arrayAcuraciasKNN.append(accuracy_score(yTest, yKNN))

In [39]:
arrayAcuraciasKNN

[0.38781575037147104,
 0.2601744186046512,
 0.31862068965517243,
 0.39367088607594936,
 0.42527173913043476,
 0.36276083467094705,
 0.29310344827586204,
 0.35973597359735976,
 0.4383116883116883,
 0.3783359497645212]

In [9]:
# O DATASET COMECA SO COM A COLUNA 0 (PASTA) E A 23 (CLASSIFICACAO)
dataset = numpy.delete(datasetOriginal, numpy.s_[1:-1], axis=1)

# ARRAY DE FEATURES QUE FALTAM, 0 E A PASTA E 23 E A CLASSIFICACAO, POR ISSO NAO ENTRAM
featuresRestantes = numpy.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])

# ENQUANTO HOUVEREM FEATURES A SEREM TESTADAS
while(len(featuresRestantes) != 0):
    
    melhorAcuraciaKNN = 0
    melhorFeatureKNN = 1
    
    # PARA CADA FEATURE RESTANTE EU A ADICIONO NAS QUE JA ESTAO SENDO UTILIZADAS
    for i, featureAtual in enumerate(featuresRestantes):
        
        # REARRANJO O DATASET COM A FEATURE ATUAL
        dataset = rearranjarDataset(dataset, datasetOriginal, featureAtual, i)
        
        # AGORA QUE JA TENHO UM DATASET COM AS FEATURES DA VEZ, TENHO QUE TESTAR OS CLASSIFICADORES
        knn = KNeighborsClassifier(7)
        arrayAcuraciasKNN = []
        
        # PARA CADA ITERACAO DO KFOLD
        matrizKFold = kFold(dataset)
        for iteracaoKFold in matrizKFold:

            # SEPARANDO OS DADOS DE TREINAMENTO E TESTE E JÁ EXCLUINDO A COLUNA 0 (pasta)
            dadosTreino = numpy.delete(iteracaoKFold[0], 0, axis=1)
            dadosTeste  = numpy.delete(iteracaoKFold[1], 0, axis=1)

            # SEPARANDO O QUE E X E Y
            xTrain = numpy.delete(dadosTreino, dadosTreino.shape[1] - 1, axis=1) # exclui a coluna do target
            xTest  = numpy.delete(dadosTeste, dadosTeste.shape[1] - 1, axis=1) # exclui a coluna do target
            yTrain = numpy.delete(dadosTreino, numpy.s_[0:dadosTreino.shape[1] - 1], axis=1).ravel() # exclui as colunas menos a 22
            yTest  = numpy.delete(dadosTeste, numpy.s_[0:dadosTeste.shape[1] - 1], axis=1).ravel() # exclui as colunas menos a 22

            # TREINANDO O CLASSIFICADOR (ignorando a pasta e usando a coluna 23 como target)
            knn = knn.fit(xTrain, yTrain)

            # PREDIZENDO
            yKNN = knn.predict(xTest)

            # VERIFICANDO A ACURACIA
            arrayAcuraciasKNN.append(accuracy_score(yTest, yKNN))
        
        # AGORA JA TENHO TODAS AS ACURACIAS DO KFOLD, VOU TIRAR A MEDIA 
        # E VER SE COM ESSA FEATURE O RESULTADO FOI BOM
        if numpy.mean(arrayAcuraciasKNN) > melhorAcuraciaKNN:
            melhorAcuraciaKNN = numpy.mean(arrayAcuraciasKNN)
            melhorFeatureKNN = featureAtual           
            
    # DEPOIS QUE ACABAR, EU REMOVO A MELHOR FEATURE DAS FEATURES RESTANTES E COLOCO ELA FIXA NO DATASET
    dataset, featuresRestantes = novaMelhorFeature(dataset, datasetOriginal, melhorFeatureKNN, featuresRestantes)

KeyboardInterrupt: 